In [1]:
# Basic Python modules
import pickle
import time
import math
import os
import re
import string
import random
import threading
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
from gensim import corpora, models, similarities
from gensim.parsing.preprocessing import STOPWORDS

#Plotting Tools
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patheffects as PathEffects
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})
#NLTK
import nltk

#Scikit learn models 
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn. feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

# Import pandas and Numpy
import pandas as pd
import numpy as np

# Chemdataextractor
from chemdataextractor.doc import Document as Doc, Heading, Paragraph

In [2]:
tag_list=[['AFM','atomic force microscop'],'electron densit',\
['SEM', 'scanning electron', 'BSE micrograph'], ['STM','Scanning tunneling'], ['band struct','band gap'],\
['XRD','X-ray diffrac','X ray diffrac'],'Strain','Raman spectr', ['TEM','Transmission electron microscop','HRTEM'],\
'Current-volt','Current volt','hysteresis','Thermodynamics', 'Network former', 'Network modifier', 'Cation', 'Anion',\
['DOS','Density of State'],'Phase diagram','Thermal conductivity', ['Scratch resistance','Scratch'],\
['Dielectric', 'Dielectric constant', 'Dielectric loss', 'breakdown', 'breakdown strength', 'BDS'],\
['TGA','Thermogravimetry'],\
['DSC','Differential scanning','Calorimetry', 'heat flow'], ['RI','Refractive ind'],\
'Cyclic volta',\
['FEM','finite element', 'FE'],['Magnetization curv', 'magnetization'], 'stress–strain curv',['plastic','ductil'],['tensi'],\
['compress'],\
'phonon',
'microstruct',\
'mass spectr', 'current map', 'isotherms',\
['resist', '3D VG theory', '3D VGT formula'], 'conduct',\
['Polarizat','PV','polarizability'], ['Glass transition temperature', 'Glass transition','Tg'], ['Crystal', 'kinetics'],\
['phase separa','immiscibility'],['Liquidus temperature', 'Liquidus', 'solidus'],\
'Charge compensator',\
['BO','Bridging'], ['NBO', 'Non-bridging '], 'Hardness', ['Fracture toughness', 'KIC'], 'Density',\
'Modulus', 'Poisson’s ratio',\
['Thermal expansion', 'CTE', 'TEC'], ['Specific heat', 'heat capacity', 'adiabatic'], \
['Structure factor', 'Neutron scattering', 'FWHM','FSDP'], 'X-ray scattering', 'Indentation', 'radiation', 'bioactive',\
['distribution function'], ['transmission', 'transmi'],\
['absorp','absorb'],['reflect'],\
['MD','molecular dynamics','molecular simulation','BKS','CHIK'],['MC', 'Monte-Carlo', 'Monte carlo'],\
['Thermo lumines', 'Thermo-luminescence', 'TL'], 'Excitation',\
['optically stimulated luminescence', 'OSL','optical lumines'], ['Electron Diffrac','Selected Area Electron Diffraction', 'Thermo luminescence', 'SAED','SEAD','SAD'],\
[ 'EDX','Energy dispersive X-ray spectroscopy', 'X-ray spectroscopy', 'EDS', 'Xray spectroscopy', 'XPS'],\
['Energy diag', 'Energy level diag', 'Energy level transition', 'Energy level scheme', 'energy diagram'],\
['PLE', 'PL', 'Photolumines'],\
'stimulated emission', ['Gain coefficient', 'Gain measurement spectra', 'Optical gain',' Gain spectra', 'Gain cross-section'], 'lifetime',\
'leakage current density', ['lumines', 'Luminescence spectra', 'luminescence decay'],\
['MAS NMR', 'Magic Angle Spinning'] , ['emission', 'emission spectra'],['DTA', 'Differential thermal'],\
['free energy', 'Helmholtz', 'absorbed energy', 'gibbs energy'], ['CIE', 'chromaticity diag'], 'Weight loss',\
['strength', 'tensile strength', 'compressive strength', 'flexural strength', 'yield strength', 'ultimate strength', 'shear strength'],\
 ['anneal','heat-treatment', 'sinter'], ['resonance', 'resonance freq’, ‘Electron paramagnetic resonance'], 'Impedance spect', \
'bonding scheme',['ionic conduc'], ['FTIR Spectr'],'IR emission',['Quantum efficiency','QE'],['iso-conversion plot'],\
['Tc','crystallization temperature','Tp'],'thermodynamic barrier',['UC emission', 'UC Luminescence', 'UC Spectra', 'UC Intensity'],'magnetic entropy', \
'energy trans', 'Cross relax', 'fracture', 'acoustic emiss', 'electric dipole',\
['NIR', 'NIR emission', 'NIR lumines','NIR fluores', 'Near Infra-red'], ['Relaxation time','relaxation'], ['IR transmi'],'quantum yield', ['Crack','Crack propagation','crack deflection'], \
'thermal sensit', 'CRET efficiency', 'spectral power distribution', 'HETCOR spect', ['UV-Vis absorp', 'Visible absorp', 'Visible spect', 'Visible emiss'], \
'fringe pattern', 'Tammann triangle',['lattice parameter', 'lattice constant', 'lattice'], 'current density',\
'FFT', 'Viscosity', \
'IR micrograph', 'Dilatometry', ['XEL spectr', 'X-ray excited luminescence spectr'], \
['Particle size distribution', 'Grain size distribution', 'PSD', 'GSD', 'Particle size', 'Grain size'], 'Optical absorp', ['activation energy','Kissinger', 'kinetics', 'Ozawa', 'JMA'], \
['CCT','correlated color temperature'], ['Eigen freq'], 'Abbe number', ['fluorescence','fluores','fluorescence intensity ratio', 'FIR'],\
['SERS Spect', 'Surface-enhanced Raman spect','SERS', 'Surface enhanced Raman spect'],'Molar Vol', 'IR fluores',\
['radio-lumines', 'RL Spect'], ['interface', 'interfac energ', 'permeation layer'],\
['VG transition line', 'VG'],\
['energy-storage density', 'storage'],\
['Qn', 'Q1', 'Q2', 'Q3', 'Q4'], ['MIR', 'MIR emission'],\
['angular', 'angular distribution'], \
'Z-scan',\
'Gruneisen', 'gamma ray',\
['Optical photograph', 'topology', 'Optical micrograph'],\
['dissolution','dissolve','flows','corros','leach','rotating disk method']]

In [3]:
def caption_labeller(DF,tag_list=tag_list):
    
    print("Commencing labeling...")
    label=0
    
    for t in tag_list:
        
        if type(t)==list:
            
            for tag in t:
            
                tag=tag.lower()
                    
                DF.loc[DF['Caption'].str.contains(tag,case=False,na=False,regex=True),'tag']=t[0]
                DF.loc[DF['Caption'].str.contains(tag,case=False,na=False,regex=True),'label']=label
        
        if type(t)==str:
            
            tag=t.lower()
             
            DF.loc[DF['Caption'].str.contains(tag,case=False,na=False,regex=True),'tag']=t
            DF.loc[DF['Caption'].str.contains(tag,case=False,na=False,regex=True),'label']=label
        
        label+=1
    
    DF.dropna(inplace=True)
    
    size_dict={}
    
    for tag in tag_list:
        if type(tag)==list:
            tag=tag[0]
        val=len(DF[DF['tag']==tag])
        size_dict[tag]=val
    
    for tag in tag_list:
        if type(tag)==list:
            tag=tag[0]
        size_dict[tag]=np.around(size_dict[tag]*20/max(list(size_dict.values()))+7,2)
    
    sorted_dict={}
    for w in sorted(size_dict, key=size_dict.get):
        sorted_dict[w]=size_dict[w]
        
    print("Labelling complete...")
            
    return sorted_dict

In [4]:
def process_data(DF, column_name):
    data = d[column_name].tolist()
    data_words = list(sent_to_words(data))
    
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100) 
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    
    data_words_nostops = [[word for word in simple_preprocess(str(doc)) if word not in STOPWORDS] for doc in data_words]
    data_words_bigrams = [bigram_mod[doc] for doc in data_words_nostops]
    
    id2word = corpora.Dictionary(data_words_bigrams)
    texts = data_words_bigrams
    corpus = [id2word.doc2bow(text) for text in texts]
    
    return corpus, id2word

In [5]:
def folder_search(Query, pii, base):
        
        pii_path=os.path.join(base,Query,pii)
        
        TempDF=pd.DataFrame(columns=['Figname','No','Caption'])
        
        if os.path.isdir(pii_path):

                cap_path=os.path.join(pii_path,pii+'-caption.txt')

                if os.path.exists(cap_path):
                    
                        TempDF=pd.read_csv(cap_path,header=None, names=['Figname','No','Caption'])
                        TempDF['Query']=Query
                        TempDF['PII']=pii
                
        
        return TempDF

In [6]:
ascii4=re.compile('\W+')
number=re.compile('\d+')
extra_space=re.compile('\s\s+')

def text_prepare(text):
    temp=text.split(' ') 
    for word in STOPWORDS:
        temp=[x for x in temp if x!=word]
    text=' '.join(temp)
    return text

def preparer(DF):

    
    text=list(DF['Caption'])
    
    new_text=[]
    for t in text:
        if type(t)==str:
            t=t.replace(' ','ABC')
            t=ascii4.sub('',t)
            t=number.sub('ABC',t)
            t=t.replace('ABC',' ')
            t=extra_space.sub(' ',t)
            new_text.append(t)
    
    prepared = []
    for line in new_text:
        line = text_prepare(line.strip())
        prepared.append(line)
        
    return prepared

def vectorizer(prepared,DF,i):
    
    print('Featurizing\n')
    min_val=100
    tfidf=TfidfVectorizer(min_df=min_val,max_df=0.9,ngram_range=(1,3))
    
    features=tfidf.fit_transform(prepared)
    names=tfidf.get_feature_names()
    
    featDF=pd.DataFrame(features.todense(),columns=tfidf.get_feature_names())
    
    print('Cosine Matrix \n')
    X=featDF.values
    distance_matrix = pairwise_distances(X, X, metric='cosine', n_jobs=-1)
    
    print('TSNE \n')
    state=random.randint(0,200)
    time_start=time.time()
    tsne=TSNE(n_components=2,metric="precomputed",verbose=1,random_state=state,perplexity=40,n_iter=300)
    tsne_results=tsne.fit_transform(distance_matrix)

    print('t-SNE done! Time.elpased: {}'.format(time.time()-time_start))
    
    DF['State_'+str(i)]=state
    DF['x-label-'+str(i)]=tsne_results[:,0]
    DF['y-label-'+str(i)]=tsne_results[:,1]
    
    return None

def ccp_scatter(DF,sorted_dict,title,i):
    
    palette = np.array(sns.color_palette("hls", len(sorted_dict)))
    
    f = plt.figure(figsize=(12, 12))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(DF['x-label-'+str(i)], DF['y-label-'+str(i)], lw=0, s=20,c=palette[DF['label'].astype(np.int)])
    ax.axis('off')
    ax.axis('tight')
    
    txts = []
    for item in sorted_dict.keys():
        # Position of each label.
        xtext, ytext = np.median(DF[DF['tag']==item][['x-label-'+str(i),'y-label-'+str(i)]],axis=0)
        if not math.isnan(xtext) and not math.isnan(ytext):
            txt = ax.text(xtext, ytext, item, fontsize=sorted_dict[item])
            txt.set_path_effects([
                PathEffects.Stroke(linewidth=5, foreground="w"),
                PathEffects.Normal()])
            txts.append(txt)
    
    plt.savefig(title+'.png')
    plt.show()
    
    return None

In [7]:
def sent_to_words(sentences):
        for sentence in sentences:
             yield(simple_preprocess(str(sentence), deacc=True)) 
        
def preprocess(text):
    result = []
    text=" ".join(text.split())
    for token in simple_preprocess(text):
        if token not in STOPWORDS and len(token) > 3:
            result.append((token))
    return result

def sim_calculator(DF, column_name):

    print("Number of {}: {}".format(column_name,len(DF[column_name])))

    #Preprocessing
    print('\nCreating Dictionary...')
    processed_docs = DF[column_name].map(preprocess)

    #Generating dictionary
    dictionary = corpora.Dictionary(processed_docs)
    dictionary.filter_extremes(no_below=100,no_above=0.9, keep_n=100000)
    print('Dictionary created')
    print("Size of vocabularly: ",len(dictionary))

    #Bag of words
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

    #Tfid vectorization
    print('\nRunning TFIDF vectorization...')
    model2 = TfidfModel(bow_corpus) 
    abs_tfidf=model2[bow_corpus]
    print('TFIDF complete')

    #Calculating similarties
    print('\nCalculating Similarity Matrix...')
    index = similarities.MatrixSimilarity(abs_tfidf)
    sims = index[abs_tfidf]
    print("size of similarity matrix: ", sims.shape)

    return sims

def tsne(sims,title,DF,i):

    #run PCA
    print("\nBeginning clustering...")
    N=100
    pca = PCA(n_components=N)
    pca_result = pca.fit_transform(sims)
    var=np.sum(pca.explained_variance_ratio_)
    print("\nPCA calculated")
    print('Cumulative explained variation for {} principal components: {}'.format(N,var))

    #Running TSNE
    print("Creating TSNE labels...")
    tsne=TSNE(n_components=2,verbose=1,perplexity=40,n_iter=300)
    tsne_results=tsne.fit_transform(pca_result)
    print("TSNE complete")
    DF['x-label-'+str(i)]=tsne_results[:,0]
    DF['y-label-'+str(i)]=tsne_results[:,1]

    return str(np.round(var*100,2))

def dictionarizer(DF,topic_num):
    
    topic_num=str(topic_num)
    size_dict={}
    tag_list=list(set(DF[topic_num+'-topic']))

    for tag in tag_list:
        val=len(DF[DF[topic_num+'-topic']==tag])
        size_dict[str(tag)]=val

    for tag in tag_list:
        size_dict[str(tag)]=np.around(size_dict[str(tag)]*20/max(list(size_dict.values()))+7,2)

    sorted_dict={}
    for w in sorted(size_dict, key=size_dict.get):
        sorted_dict[w]=size_dict[w]

    return sorted_dict

def lda_scatter(DF,sorted_dict,topic_num,i,title,session_id,var='62.3'):
    
    sample_size=len(DF)
    
    topic_num=str(topic_num)
    print("\nCreating LDA plot...")

    if len(DF)>=50000:
        s=10
    else:
        s=40

    palette = np.array(sns.color_palette("hls", len(sorted_dict)+5))

    f = plt.figure(figsize=(12, 12))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(DF['x-label-'+str(i)], DF['y-label-'+str(i)], s,lw=0,c=palette[DF[topic_num+'-topic'].astype(np.int)])
    ax.axis('off')
    ax.axis('tight')

    txts = []
    for item in sorted_dict.keys():
        # Position of each label.
        xtext, ytext = np.median(DF[DF[topic_num+'-topic']==float(item)][['x-label-'+str(i),'y-label-'+str(i)]],axis=0)
        if not math.isnan(xtext) and not math.isnan(ytext):
            txt = ax.text(xtext, ytext, item, fontsize=sorted_dict[item])
            txt.set_path_effects([
                PathEffects.Stroke(linewidth=5, foreground="w"),
                PathEffects.Normal()])
            txts.append(txt)

    PCA_text='PCA: 100 \nTotal Variance: '+var

    plt.title('Cluster plot of '+str(sample_size)+' Abstracts with '+topic_num+' topics')
    plt.annotate(PCA_text, xy=(0,0), xytext=(12, 80), va='top',xycoords='axes fraction', textcoords='offset points')
    plt.savefig(title+'_'+session_id+'.png')
    print("\nLDA plot saved as {} at {}".format(title+'.png',os.getcwd()))
    plt.show()
    
    return None

In [52]:
def chem_finder(index,DF):
    
    doc=Doc(ddd.loc[index,'Abstract_x'])
    records=doc.records.serialize()
    
    try:
        chem_rec='_'
        for rec in records:
            
            if 'names' in str(rec.keys()):
                chem_rec+=', '+rec['names'][0]
            else:
                pass
            
        ddd.loc[index,'Chemicals']=chem_rec
        
    except:
        return None
    
    return None



def chemicaller(DF, threads=10):
    
    Max=len(DF)
    
    DF['Chemicals']=None
    
    for i in range(0,Max,threads):
        
        if i+threads>Max:
            temp=Max%threads
        else: 
            temp=threads
        
        parserThreads=[]
        
        for r in range(0,temp):
            
            parserThread=threading.Thread(target=chem_finder,args=((i+r),))
            parserThreads.append(parserThread)    
            parserThread.start()
                
        for parserThread in parserThreads:
            parserThread.join()
            
    return None 

In [53]:
chemicaller(D.DF[:100])

/home/civil/visitor/vineethv.visitor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Exception in thread Thread-705:
Traceback (most recent call last):
  File "/home/civil/visitor/vineethv.visitor/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/civil/visitor/vineethv.visitor/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
TypeError: chem_finder() missing 1 required positional argument: 'DF'

Exception in thread Thread-706:
Traceback (most recent call last):
  File "/home/civil/visitor/vineethv.visitor/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
 

In [47]:
D.DF.head(10)

,PII,Title_x,Abstract_x,Query,DOI,Date,Journal,Keywords,Images,Sections,Chemicals
12967,S2314808X16000166,In vitro study of aqueous leaf extract of Chen...,The leaves of Chenopodium album Linn. are trad...,Glass And Crystallization,doi:10.1016/j.ejbas.2016.02.001,2016-03-02,Egyptian Journal of Basic and Applied Sciences,"['Bathua', 'Urolithiasis', 'Cystone', 'Calcium...",6,6,None
52420,S1002072117609979,A novel Ce(IV) ion-selective polyvinyl chlorid...,A novel Ce(IV) ion-selective polyvinyl chlorid...,Glass AND Rare earth,doi:10.1016/S1002-0721(17)60997-9,2017-09-19,Journal of Rare Earths,"['Ce(IV) ion-selective electrode', 'HDEHP', 'H...",0,0,None
4504,S0928098717301501,Innovative approach for identifying root cause...,"In sterile drug product manufacturing, scratch...",glass AND Durability,doi:10.1016/j.ejps.2017.03.015,2017-03-14,European Journal of Pharmaceutical Sciences,"['Process Analytical Technology (PAT)', 'Quali...",0,0,None
7350,S0022309397000185,Direct observation of time-resolved excited st...,The time-resolved excited state absorption spe...,Tellurite glasses,doi:10.1016/S0022-3093(97)00018-5,1999-03-11,Journal of Non-Crystalline Solids,[],0,0,None
65425,S0032591018310921,Green nanosilica@folic Acid (VB9) nanocomposit...,Folic acid (VB9) was used as an eco-friendly s...,Glass powder,doi:10.1016/j.powtec.2018.12.055,2018-12-13,Powder Technology,"['Folic acid', 'Vitamin B9', 'Nanosilica', 'Bi...",0,0,None
10664,S0921452614006899,Thermoluminescence studies on single crystal p...,"Undoped, Mn doped and Cu doped samples of lith...",Glass AND Thermoluminescence,doi:10.1016/j.physb.2014.08.023,2014-08-27,Physica B: Condensed Matter,"['Thermoluminescence', 'Lithium tetraborate', ...",3,6,None
36392,S037596010701599X,Dielectric response in ferroelectric BaTiO3,The far-infrared optical and dielectric proper...,Glass AND Dielectric properties,doi:10.1016/j.physleta.2007.11.009,2007-11-12,Physics Letters A,"['Perovskite', 'Barium compounds', 'Optical ph...",0,0,None
6636,001670379390606W,Noble gas solubilities in silicate melts and g...,New measurements of the solubility of Ar in ba...,Tellurite glasses,doi:10.1016/0016-7037(93)90606-W,2003-03-27,Geochimica et Cosmochimica Acta,[],0,0,None
85015,B9780080168746500783,SOLVENT EFFECTS ON NADH FLUORESCENCE IN SOLUTI...,Publisher Summary This chapter discusses about...,Glass AND Structure,doi:10.1016/B978-0-08-016874-6.50078-3,2014-06-27,Structure and Function of Oxidation–Reduction ...,[],0,0,None
20350,S0040403914000963,Investigating the effects of side chain length...,The emissive properties of fluorophores in agg...,Glass properties,doi:10.1016/j.tetlet.2014.01.062,2014-01-23,Tetrahedron Letters,"['Tetraphenylethylene (TPE)', 'Aggregation-ind...",7,4,None


In [39]:
chemicaller(D.DF[:100])

/home/civil/visitor/vineethv.visitor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Exception in thread Thread-404:
Traceback (most recent call last):
  File "/home/civil/visitor/vineethv.visitor/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/civil/visitor/vineethv.visitor/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
TypeError: chem_finder() takes 1 positional argument but 11 were given

Exception in thread Thread-405:
Traceback (most recent call last):
  File "/home/civil/visitor/vineethv.visitor/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()


In [31]:
ddd=D.DF.iloc[3]
doc=Doc(ddd.loc['Abstract_x'])
records=doc.records.serialize()

try:
    chem_rec='_'
    for rec in records:
        
        if 'names' in str(rec.keys()):
            chem_rec+=', '+rec['names'][0]
        else:
            pass
        
    ddd.loc['Chemicals']=chem_rec
    
except:
    None
        

In [27]:
#ddd=D.DF.iloc[1]
D.DF.iloc[1]

Index(['PII', 'Title_x', 'Abstract_x', 'Query', 'DOI', 'Date', 'Journal',
       'Keywords', 'Images', 'Sections'],
      dtype='object')

In [14]:
class Document:
    
    def __init__(self,address):
        
        self.session_id=self.randomString()
        
        if type(address)==str:
            self.DF=pd.read_csv(address)
        elif isinstance(DF, pd.DataFrame):
            self.DF=address
        
        self.size=len(self.DF)
        
        self.lda_model=[]
        
        self.caption_DF=None
        self.caption_labels_dictionary=None
        
        
    def randomString(self,stringLength=8):
        lettersAndDigits = string.ascii_letters + string.digits
        return ''.join(random.choice(lettersAndDigits) for i in range(stringLength))
    
    def generate_Captions(self,PII='PII',Query='Query',base=os.getcwd()):
        Cap_list=[]
        for row in self.DF.index:
            pii=self.DF.loc[row,PII]
            Q=self.DF.loc[row,Query]
            TempDF=folder_search(Q,pii,base)
            Cap_list.append(TempDF)
            
        self.caption_DF=pd.concat(Cap_list)
        
        print("Caption generation complete. \nNo of Captions found : {}".format(len(self.caption_DF)))
            
        return None
    
    def plot_caption_cluster(self, iterations=1):
        prepared=preparer(self.caption_DF)
        
        for i in range(iterations):
            TDF=vectorizer(prepared,self.caption_DF,i) 
            title='Caption cluster plot of '+str(len(self.caption_DF))+'Captions-'+str(i)
            ccp_scatter(self.caption_DF,self.caption_labels_dictionary,title,i)
            
        return None
        
    def label_Captions(self,tag_list=tag_list):
        
        if isinstance(self.caption_DF, pd.DataFrame):
            self.caption_labels_dictionary=caption_labeller(self.caption_DF,tag_list=tag_list)
        else:
            print("No captions found. Run DF.generate_Captions()")
        
        return None
    
    def run_LDA(self,column_name, topic_num, passes=10):
        
        print("Preparing data to run LDA...")
        corpus, id2word=process_data(self.DF, column_name)

        print('Running LDA...')

        start=time.time()
        lda_model = gensim.models.LdaMulticore(corpus,num_topics=topic_num, id2word=id2word, passes=passes, workers=None,
                                               chunksize=1000)
        stop=time.time()

        print("LDA complete")
        print("Total time: {}".format(stop-start))

        print("\nStoring data in dataframe")
        
        self.lda_model.append((topic_num,lda_model, self.session_id))
        

        for i,j in enumerate(self.DF.index):
            index, score = sorted(lda_model[corpus[i]], key=lambda tup: -1*tup[1])[0]
            self.DF.loc[j,str(topic_num)+'-topic']=index
            self.DF.loc[j,str(topic_num)+'-topic-score']=score

        print("Done")
        
        return None
        
    
    def generate_LDA_plot(self,column_name,topic_num, repeats=1, pca=True):
        
        if len(self.lda_model)>0:
            
            for i in range(repeats):
                column_name='Abstract_x'
                title='LDA Plot for '+str(self.size)+' '+column_name+' for '+str(topic_num)+'topics-'+str(i)
                sims=sim_calculator(self.DF, column_name)
                var=tsne(sims,topic_num,self.DF,i)
                sorted_dict=dictionarizer(self.DF,topic_num)
                lda_scatter(d,sorted_dict,topic_num,i,title,self.session_id,var)
        
        else:
            print("No lda models found. Please run DF.run_LDA()")
                
        return None

In [11]:
DF=pd.read_csv('Revised_Glass_Database.csv')

In [12]:
d=DF.sample(1000)

In [15]:
D=Document(d)

In [16]:
D.session_id

'zGemYc5E'

In [17]:
D.size

1000

In [ ]:
D.generate_Captions(PII='PII',Query='Query')

In [ ]:
D.label_Captions()

In [ ]:
D.plot_caption_cluster()

In [ ]:
D.run_LDA('Abstract_x',15)

In [ ]:
D.generate_LDA_plot('Abstract_x',15)